In [1]:
! nvidia-smi

Mon Apr 29 00:50:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              25W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## **Finetunning the llava model(VSFT on the llava-hf/llava-1.5-7b-hf model with 260k image and conversation pairs from the HuggingFaceH4/llava-instruct-mix-vsft dataset.) with custom dataset**

In [2]:
%%capture
!pip install -U "transformers>=4.39.0"
!pip install peft bitsandbytes
!pip install -U "trl>=0.8.3"
!pip install deep_translator

In [3]:
import torch
from transformers import AutoTokenizer, AutoProcessor, TrainingArguments, LlavaForConditionalGeneration, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer


2024-04-29 01:49:55.284039: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 01:49:55.284163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 01:49:55.418581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# model_id = "llava-hf/llava-1.5-7b-hf"
model_id = "HuggingFaceH4/vsft-llava-1.5-7b-hf-trl"

In [5]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

In [6]:
model = LlavaForConditionalGeneration.from_pretrained(model_id,
                                                      quantization_config=quantization_config,
                                                      torch_dtype=torch.float16)

config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [7]:
LLAVA_CHAT_TEMPLATE = """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. {% for message in messages %}{% if message['role'] == 'user' %}USER: {% else %}ASSISTANT: {% endif %}{% for item in message['content'] %}{% if item['type'] == 'text' %}{{ item['text'] }}{% elif item['type'] == 'image' %}<image>{% endif %}{% endfor %}{% if message['role'] == 'user' %} {% else %}{{eos_token}}{% endif %}{% endfor %}"""

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.chat_template = LLAVA_CHAT_TEMPLATE
processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer = tokenizer

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from PIL import Image
import requests
import numpy as np
import ast
import json

class LLavaDataCollator:
    def __init__(self, processor, image_size=(640,480)):
        self.processor = processor
        self.image_size = image_size

#     def resize_image(self, image_path):
#         image = Image.open(image_path)
#         image = image.resize(self.image_size, Image.ANTIALIAS)
#         return image

    def __call__(self, examples):
        texts = []
        images = []
        for example in examples:
#             messages = [{'content': [{'index': None, 'text': 'How many computer monitors are on the wooden desk?\n', 'type': 'text'}, {'index': 0, 'text': None, 'type': 'image'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'There are two computer monitors on the wooden desk.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': 'Can you describe the position of the chair and the person in relation to the wooden desk?', 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'The person is sitting in the chair in front of the wooden desk, using a laptop computer. Their feet are up on the desk, and there are other computers in the background.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': 'What is the person doing with the laptop computer?', 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'The person is using the laptop, possibly working or engaging in some recreational activity. They appear to be in a relaxed, lounging position with their feet up on the desk.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': 'How many computers can be seen in the image? Is there an additional computer aside from the laptop?', 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'There are a total of three computers visible in the image. These include the laptop being used by the person, and two additional monitors on the wooden desk.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': "Considering the presence of the laptop computer and multiple monitors, what could be the possible implications for the person's work or personal preferences?", 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': "Given the presence of the laptop computer and multiple additional monitors, it is likely that the person either has a demanding job or tasks that require them to multitask across multiple screens, or they have a personal preference for an extended workspace to enhance their productivity or workflow.\n\nHaving multiple monitors can help improve efficiency and work quality in various ways, such as managing multiple open applications, web pages, and documents simultaneously without having to constantly switch back and forth between them on a single screen. People working in fields like programming, design, video editing, gaming, or finance might especially benefit from this setup, as it allows them to access and compare different sets of data or work on multiple tasks at the same time with ease.\n\nAdditionally, having a laptop computer in addition to the multiple monitors could indicate a need for mobility or flexibility in the person's work environment. They might travel frequently or work remotely, in which case a laptop would be essential. Having a laptop on hand also provides a convenient backup device should any technical issues arise with the other computers.\n\nAt a personal level, the person might simply enjoy the versatility and workspace expansion that comes with using a laptop alongside multiple monitors. For someone who spends a significant amount of time in front of screens, be it for work, communication, or leisure activities, such a setup can help create a comfortable and efficient workspace that caters to their needs and preferences.", 'type': 'text'}], 'role': 'assistant'}]
#             messages = {'content': [{'index': None, 'text': 'How many computer monitors are on the wooden desk?\n', 'type': 'text'}, {'index': 0, 'text': None, 'type': 'image'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'There are two computer monitors on the wooden desk.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': 'Can you describe the position of the chair and the person in relation to the wooden desk?', 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'The person is sitting in the chair in front of the wooden desk, using a laptop computer. Their feet are up on the desk, and there are other computers in the background.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': 'What is the person doing with the laptop computer?', 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'The person is using the laptop, possibly working or engaging in some recreational activity. They appear to be in a relaxed, lounging position with their feet up on the desk.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': 'How many computers can be seen in the image? Is there an additional computer aside from the laptop?', 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': 'There are a total of three computers visible in the image. These include the laptop being used by the person, and two additional monitors on the wooden desk.', 'type': 'text'}], 'role': 'assistant'}, {'content': [{'index': None, 'text': "Considering the presence of the laptop computer and multiple monitors, what could be the possible implications for the person's work or personal preferences?", 'type': 'text'}], 'role': 'user'}, {'content': [{'index': None, 'text': "Given the presence of the laptop computer and multiple additional monitors, it is likely that the person either has a demanding job or tasks that require them to multitask across multiple screens, or they have a personal preference for an extended workspace to enhance their productivity or workflow.\n\nHaving multiple monitors can help improve efficiency and work quality in various ways, such as managing multiple open applications, web pages, and documents simultaneously without having to constantly switch back and forth between them on a single screen. People working in fields like programming, design, video editing, gaming, or finance might especially benefit from this setup, as it allows them to access and compare different sets of data or work on multiple tasks at the same time with ease.\n\nAdditionally, having a laptop computer in addition to the multiple monitors could indicate a need for mobility or flexibility in the person's work environment. They might travel frequently or work remotely, in which case a laptop would be essential. Having a laptop on hand also provides a convenient backup device should any technical issues arise with the other computers.\n\nAt a personal level, the person might simply enjoy the versatility and workspace expansion that comes with using a laptop alongside multiple monitors. For someone who spends a significant amount of time in front of screens, be it for work, communication, or leisure activities, such a setup can help create a comfortable and efficient workspace that caters to their needs and preferences.", 'type': 'text'}], 'role': 'assistant'}]
#             m = example["messages"]
#             d = json.loads(m)
#             messages = [ { "content": [ { "index": null, "text": "Who wrote this book?\n", "type": "text" }, { "index": 0, "text": null, "type": "image" } ], "role": "user" }, { "content": [ { "index": null, "text": "Blue Star Coloring", "type": "text" } ], "role": "assistant" }, { "content": [ { "index": null, "text": "What is the title of this book?", "type": "text" } ], "role": "user" }, { "content": [ { "index": null, "text": "Adult Coloring Book: Stress Relieving Animal Designs Volume 2", "type": "text" } ], "role": "assistant" }, { "content": [ { "index": null, "text": "What is the genre of this book?", "type": "text" } ], "role": "user" }, { "content": [ { "index": null, "text": "Humor & Entertainment", "type": "text" } ], "role": "assistant" }, { "content": [ { "index": null, "text": "Is this a comedy book?", "type": "text" } ], "role": "user" }, { "content": [ { "index": null, "text": "Yes", "type": "text" } ], "role": "assistant" }, { "content": [ { "index": null, "text": "Is this a historical book?", "type": "text" } ], "role": "user" }, { "content": [ { "index": null, "text": "No", "type": "text" } ], "role": "assistant" } ]
#             messages.append(d)
#             messages = [{"content": [{"index": null, "text": "Who painted this portrait?", "type": "text"}, {"index": 0, "text": null, "type": "image"}], "role": "user"}, {"content": [{"index": null, "text": "John Singleton Copley", "type": "text"}], "role": "assistant"}, {"content": [{"index": null, "text": "When was this portrait created?", "type": "text"}], "role": "user"}, {"content": [{"index": null, "text": "Between 1760 and 1770", "type": "text"}], "role": "assistant"}, {"content": [{"index": null, "text": "Identify the technique used in the creation of this portrait.", "type": "text"}], "role": "user"}, {"content": [{"index": null, "text": "Oil on canvas", "type": "text"}], "role": "assistant"}, {"content": [{"index": null, "text": "What is the name of the subject featured in the portrait?", "type": "text"}], "role": "user"}, {"content": [{"index": null, "text": "Nathaniel Hurd", "type": "text"}], "role": "assistant"}, {"content": [{"index": null, "text": "Describe the artistic style of this portrait.", "type": "text"}], "role": "user"}, {"content": [{"index": null, "text": "The style of the portrait is 18th-century American painting.", "type": "text"}], "role": "assistant"}]
            
            messages = example["messages"]
#             print(type(messages))
#             print(messages)
            text = self.processor.tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=False
            )
            texts.append(text)
#             url = "https://www.ilankelman.org/stopsigns/australia.jpg"
#             image = Image.open(requests.get(url, stream=True).raw)
#             print(example["images"]) 
            image = Image.open(example["image"])
#             image = Image.open("/kaggle/input/images/images/1915.534_web.jpg")
#             resized_image = self.resize_image(example["images"])
#             print(example["images"][0])
#             images.append(example["images"][0])
#             print(image)
            images.append(image)

        batch = self.processor(texts, images, return_tensors="pt", padding=True)
#         print(batch)
        labels = batch["input_ids"].clone()
#         print(labels)
        if self.processor.tokenizer.pad_token_id is not None:
            labels[labels == self.processor.tokenizer.pad_token_id] = -100
        batch["labels"] = labels
#         print(batch)
        return batch

# Example usage:
# processor = ...  # Initialize your processor
data_collator = LLavaDataCollator(processor)


### **Importing Training and Test Dataset**

In [10]:
from datasets import load_dataset
train_dataset_2 = load_dataset("/kaggle/input/train-5")
test_dataset_2 = load_dataset("/kaggle/input/test-5")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
train_dataset_1 = train_dataset_2["train"]
# specific_train_dataset = train_dataset_2["train"]
# train_dataset_1 = specific_train_dataset.shuffle(seed=42).select(range(40))
specific_test_dataset = test_dataset_2["test"]
eval_dataset_1 = specific_test_dataset.shuffle(seed=42).select(range(2))

In [16]:
training_args = TrainingArguments(
    output_dir="llava-1.5-7b-hf-ft-mix-vsft-3",
    report_to="tensorboard",
    learning_rate=1.4e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    logging_steps=5,
    num_train_epochs=1,
    push_to_hub=True,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    fp16=True,
    bf16=False
)

In [17]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules="all-linear"
)

In [14]:
%%capture
!pip install huggingface_hub

In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [18]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_1,
    eval_dataset=eval_dataset_1,
    peft_config=lora_config,
    dataset_text_field="text",  # need a dummy field
    tokenizer=tokenizer,
    data_collator=data_collator,
    dataset_kwargs={"skip_prepare_dataset": True},
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


### **Training**

In [19]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_

Step,Training Loss


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=3, training_loss=0.9628692468007406, metrics={'train_runtime': 35.417, 'train_samples_per_second': 0.169, 'train_steps_per_second': 0.085, 'total_flos': 54532828078080.0, 'train_loss': 0.9628692468007406, 'epoch': 1.0})

### **Pushing to Hub**

In [20]:
trainer.push_to_hub()

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1714353876.6e9d271be7ad.33.0:   0%|          | 0.00/5.87k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Praveen0309/llava-1.5-7b-hf-ft-mix-vsft-3/commit/6188f30e2f215b54c428d2f5b6ce43f69eeae048', commit_message='End of training', commit_description='', oid='6188f30e2f215b54c428d2f5b6ce43f69eeae048', pr_url=None, pr_revision=None, pr_num=None)

### **Inferring from Hub**

In [21]:
from peft import PeftConfig, PeftModel
from PIL import Image
import requests

# model_id = "llava-hf/llava-1.5-7b-hf"
model_id = "HuggingFaceH4/vsft-llava-1.5-7b-hf-trl"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

base_model = LlavaForConditionalGeneration.from_pretrained(model_id,
                                                      quantization_config=quantization_config,
                                                      torch_dtype=torch.float16)
# Load the PEFT Lora model (adapter)
peft_lora_adapter_path = "Praveen0309/llava-1.5-7b-hf-ft-mix-vsft-3"
peft_lora_adapter = PeftModel.from_pretrained(base_model, peft_lora_adapter_path, adapter_name="lora_adapter")

# Merge the adapters into the base model
base_model.load_adapter(peft_lora_adapter_path, adapter_name="lora_adapter")

# Now you can use the combined model (base model + PEFT Lora adapter) for inference
# For example:
prompt = "USER: <image>\nWhat's the content of the image? ASSISTANT:"
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)
# processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor = AutoProcessor.from_pretrained("HuggingFaceH4/vsft-llava-1.5-7b-hf-trl")
inputs = processor(text=prompt, images=image, return_tensors="pt")
# ... process the image and create inputs ...
generate_ids = base_model.generate(**inputs, max_new_tokens=15)
decoded_response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print("Generated response:", decoded_response)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/941 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated response: USER:  
What's the content of the image? ASSISTANT: The image features a stop sign on a city street, with a car driving


### **Saving model locally**

In [22]:
base_model.save_pretrained("kaggle_model_3")

/opt/conda/lib/python3.10/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


### **Inferring Locally**

In [4]:
from peft import PeftConfig, PeftModel
from PIL import Image
import requests

# model_id = "llava-hf/llava-1.5-7b-hf"
model_id = r"C:\Users\prave\OneDrive\Desktop\MLOPS\Mlops_2\huggingface_model"
base_model = LlavaForConditionalGeneration.from_pretrained(model_id)

processor = AutoProcessor.from_pretrained(r"C:\Users\prave\OneDrive\Desktop\MLOPS\Mlops_2\huggingface_processor")

# Load the PEFT Lora model (adapter)
peft_lora_adapter_path = r"C:\Users\prave\OneDrive\Desktop\MLOPS\Mlops_2\huggingface_adapter"

# Load the PEFT Lora model (adapter)
peft_lora_adapter = PeftModel.from_pretrained(base_model, peft_lora_adapter_path, adapter_name="lora_adapter")

# Merge the adapters into the base model
base_model.load_adapter(peft_lora_adapter_path, adapter_name="lora_adapter")

# Now you can use the combined model (base model + PEFT Lora adapter) for inference
# For example:
prompt = "USER: <image>\nWhat's the content of the image? ASSISTANT:"
# url = "https://www.ilankelman.org/stopsigns/australia.jpg"
url = "/kaggle/input/images/images/1921.428_web.jpg"
image = Image.open(url)
# image = Image.open(requests.get(url, stream=True).raw)
# # processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
inputs = processor(text=prompt, images=image, return_tensors="pt")
# ... process the image and create inputs ...
generate_ids = base_model.generate(**inputs, max_new_tokens=15)
decoded_response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print("Generated response:", decoded_response)

config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Generated response: USER:  
What's the content of the image? ASSISTANT: The image features a stop sign on a city street, with a car driving


### **Translating**

In [6]:
from deep_translator import GoogleTranslator
def deep_translator_bn_en(input_sentence):
  english_translation = GoogleTranslator(source="bn", target="en").translate(input_sentence)
  return english_translation

def deep_translator_en_bn(input_sentence):
  bengali_translation = GoogleTranslator(source="en", target="bn").translate(input_sentence)
  return bengali_translation


In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def facebook_response(url, input_sentence):
  url = input("ইমেজ url লিখুন: ")
  input_sentence = input("ছবি সম্পর্কে আপনার প্রশ্ন লিখুন: ")
  image_prompt = facebook_bn_en(input_sentence)
  response = inference(image_prompt, url)
  assistant_index = response.find("ASSISTANT:")
  extracted_string = response[assistant_index + len("ASSISTANT:"):].strip()
  output = facebook_en_bn(extracted_string)
  print("বটী: ", output)
  return output


In [14]:
def inference(image_prompt, url):
    prompt = f"USER: <image>\n{image_prompt} ASSISTANT:"
#     prompt = "USER: <image>\nWhat's the content of the image? ASSISTANT:"
    # url = "https://www.ilankelman.org/stopsigns/australia.jpg"
#     url = "/kaggle/input/images/images/1921.428_web.jpg"
    # image = Image.open(url)
    image = Image.open(requests.get(url, stream=True).raw)
    # processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    # ... process the image and create inputs ...
    generate_ids = base_model.generate(**inputs, max_new_tokens=15)
    decoded_response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#     print("Generated response:", decoded_response)
    return decoded_response


In [18]:
url = "/kaggle/input/images/images/1921.428_web.jpg"
input_sentence = "ছবির বিষয়বস্তু কি?"
image_prompt = deep_translator_bn_en(input_sentence)
response = inference(image_prompt, url)
assistant_index = response.find("ASSISTANT:")
extracted_string = response[assistant_index + len("ASSISTANT:"):].strip()
output = deep_translator_en_bn(extracted_string)
print(output)

ছবিতে সাদা পোশাক পরা একজন মহিলা চেয়ারে বসে আছেন


In [21]:
url = input("ইমেজ url লিখুন: ")
input_sentence = input("ছবি সম্পর্কে আপনার প্রশ্ন লিখুন: ")
image_prompt = deep_translator_bn_en(input_sentence)
response = inference(image_prompt, url)
assistant_index = response.find("ASSISTANT:")
extracted_string = response[assistant_index + len("ASSISTANT:"):].strip()
output = deep_translator_en_bn(extracted_string)
print("বটী: ", output)

ইমেজ url লিখুন /kaggle/input/images/images/1921.428_web.jpg
ছবি সম্পর্কে আপনার প্রশ্ন লিখুন ছবির বিষয়বস্তু কি?


বটী:  ছবিতে সাদা পোশাক পরা একজন মহিলা চেয়ারে বসে আছেন


In [ ]:
url = input("ইমেজ url লিখুন: ")
input_sentence = input("ছবি সম্পর্কে আপনার প্রশ্ন লিখুন: ")
image_prompt = facebook_translator_bn_en(input_sentence)
response = inference(image_prompt, url)
assistant_index = response.find("ASSISTANT:")
extracted_string = response[assistant_index + len("ASSISTANT:"):].strip()
output = facebook_translator_en_bn(extracted_string)
print("বটী: ", output)